### Basic subject-level

Let's try to imitate what we've already done at the subject level.

In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *
import gc
import nibabel as nib

nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
test_train_df = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")


dataset_name = 'conditions'
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_' + dataset_name + '_40subs.pkl'

with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)
    
    


from nilearn.decoding import DecoderRegressor, Decoder

script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'
data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/"
# HRF 2s



python initialized for apply_loocv_and_save


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


4


In [2]:

def condition_resp_trans_func(X):
    return(X.condition_label)


### Repeating test measuring precision

In [28]:
from nilearn.decoding import DecoderRegressor  

def apply_single_fit_and_save(
    results_filepath,
    brain_data_filepath = '../data/Brain_Data_2sns_60subs.pkl',
    train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv",
    subjs_to_use = None, #set this to get a subset, otherwise use all of them.
    response_transform_func = None,
    decoder = None,
    mask=None,
    cpus_to_use=1
    ):
    
    preprocessed_data = load_and_preprocess(brain_data_filepath,train_test_markers_filepath,subjs_to_use,response_transform_func)

    train_y = preprocessed_data['y']
    train_X = preprocessed_data['X']
    train_groups = preprocessed_data['groups']

    if decoder is None:
        decoder = decoderConstructor(
                standardize= True,cv=LeaveOneGroupOut(),
                mask=mask,
                n_jobs=cpus_to_use,verbose=1)

    print("starting LeaveOneOut")
    #in this design, we're actually dealing with groups
    #we select group IDs and then grab the subjects
    #so we don't need to use LeaveOneGroupOut
    #the grouping is implicit

    print('Fitting decoder')
    decoder.fit(
    y=train_y,X=train_X,groups=train_groups)

    print("predicting")
    train_score = decoder.score(train_X,train_y)
    train_y_pred = decoder.predict(train_X)

    if type(decoder)==DecoderRegressor:
        weight_img = decoder.coef_img_['beta']
    else:
        weight_img = decoder.coef_img_

    print('finished learning')

    print("saving")

    result_package = {
            'decoder':decoder,
            'weight_img':weight_img,
            'obs':train_y,
            'train_train_score':train_score,
            'train_y_pred': train_y_pred,
            'metadata':preprocessed_data['metadata']
        }
    with open(results_filepath, 'wb') as handle:
        pickle.dump(result_package,handle)
        
    return(result_package)


In [4]:

from sklearn.metrics import recall_score
from sklearn.metrics import get_scorer

from  https://nilearn.github.io/modules/generated/nilearn.decoding.Decoder.html

here's how to define a callable scorer:

scoring: str, callable or None, optional. Default: ‘roc_auc’

The scoring strategy to use. See the scikit-learn documentation at https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules If callable, takes as arguments the fitted estimator, the test data (X_test) and the test target (y_test) if y is not None. e.g. scorer(estimator, X_test, y_test)


In [59]:
def recall_multiclass_score(estimator,X_test,y_test):
    y_pred = estimator.predict(X_test)
    print("y_pred:")
    print(y_pred)
    print("y_test:")
    print(y_test)
    return(recall_score(y_test,y_pred,average='micro'))
    #return(1)


#recall_multiclass_score(decoder,[0,1,1],[0,0,1])

## test accuracy score

In [9]:
#set vars for the function

brain_data_filepath = brain_data_filepath
train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv"
subjs_to_use = 10
response_transform_func=condition_resp_trans_func
mask=None

results_filepath=data_path + "SST/train_test_results_recall_" + dataset_name + "_40subs.pkl"

### custom accuracy score

In [11]:
from sklearn.metrics import accuracy_score
def custom_accuracy_score(estimator,X_test,y_test):
    #aim is to emulate the results of the default accuracy measure
    y_pred = estimator.predict(X_test)
    print("y_pred:")
    print(y_pred)
    print("y_test:")
    print(y_test)
    return(accuracy_score(y_test,y_pred))
    #return(1)


#recall_multiclass_score(decoder,[0,1,1],[0,0,1])

In [17]:
decoder_custom_accuracy = Decoder(scoring=custom_accuracy_score,
                         standardize= True,
                         cv=LeaveOneGroupOut(),verbose=1,n_jobs=10)


In [29]:
res_custom_accuracy = apply_single_fit_and_save(
    results_filepath =results_filepath,
    brain_data_filepath = brain_data_filepath,
    train_test_markers_filepath = train_test_markers_filepath,
    subjs_to_use = subjs_to_use,
    response_transform_func = condition_resp_trans_func,
    decoder = decoder_custom_accuracy
)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 40 subjects overlapping between the subjects marked for train data and the training dump file itself.
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 139
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    183
Name: condition_label, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


183
183
using 10 subjects
starting LeaveOneOut
Fitting decoder
[NiftiMasker.fit] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.fit] Computing the mask
[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


[NiftiMasker.transform_single_imgs] Cleaning extracted signals


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   10.2s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:   17.8s finished


predicting
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
y_pred:
['CorrectGo' 'CorrectStop' 'FailedStop' 'Cue' 'FailedGo' 'CorrectGo'
 'CorrectStop' 'FailedStop' 'Cue' 'FailedGo' 'CorrectGo' 'FailedStop'
 'Cue' 'CorrectGo' 'CorrectStop' 'FailedStop' 'Cue' 'FailedGo' 'CorrectGo'
 'FailedStop' 'Cue' 'FailedGo' 'CorrectGo' 'CorrectStop' 'FailedStop'
 'Cue' 'FailedGo' 'CorrectGo' 'CorrectStop' 'FailedStop' 'Cue' 'CorrectGo'
 'CorrectStop' 'FailedStop' 'Cue' 'FailedGo' 'CorrectGo' 'CorrectStop'
 'FailedStop' 'Cue' 'FailedGo' 'CorrectGo' 'CorrectStop' 'FailedStop'
 'Cue' 'FailedGo']
y_test:
0       CorrectGo
1     CorrectStop
2      FailedStop
3             Cue
4      

### default accuracy score

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
decoder_accuracy_default = Decoder(scoring='accuracy',
                         standardize= True,
                         cv=LeaveOneGroupOut(),verbose=1,n_jobs=10)


In [32]:
res_default_accuracy = apply_single_fit_and_save(
    results_filepath =results_filepath,
    brain_data_filepath = brain_data_filepath,
    train_test_markers_filepath = train_test_markers_filepath,
    subjs_to_use = subjs_to_use,
    response_transform_func = condition_resp_trans_func,
    decoder = decoder_accuracy_default
)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 40 subjects overlapping between the subjects marked for train data and the training dump file itself.
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
FailedStop     40
Cue            40
CorrectGo      40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 139
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    183
Name: condition_label, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


183
183
using 10 subjects
starting LeaveOneOut
Fitting decoder
[NiftiMasker.fit] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.fit] Computing the mask
[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    9.5s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:   17.9s finished


predicting
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
finished learning
saving


## Result

In [43]:
res_custom_accuracy['decoder'].cv_scores_

{'CorrectGo': [0.4, 0.4, 1.0, 0.4, 0.5, 0.8, 1.0, 0.2, 0.2, 0.8],
 'CorrectStop': [0.2, 0.2, 0.0, 0.2, 0.0, 0.2, 0.25, 0.2, 0.2, 0.2],
 'Cue': [0.2, 0.2, 0.3333333333333333, 0.2, 0.25, 0.2, 0.25, 0.2, 0.2, 0.2],
 'FailedGo': [0.2,
  0.2,
  0.3333333333333333,
  0.2,
  0.25,
  0.2,
  0.25,
  0.2,
  0.2,
  0.6],
 'FailedStop': [0.2,
  0.2,
  0.3333333333333333,
  0.2,
  0.25,
  0.2,
  0.25,
  0.2,
  0.2,
  0.2]}

In [44]:
res_default_accuracy['decoder'].cv_scores_

{'CorrectGo': [0.4, 0.4, 1.0, 0.4, 0.5, 0.8, 1.0, 0.2, 0.2, 0.8],
 'CorrectStop': [0.2, 0.2, 0.0, 0.2, 0.0, 0.2, 0.25, 0.2, 0.2, 0.2],
 'Cue': [0.2, 0.2, 0.3333333333333333, 0.2, 0.25, 0.2, 0.25, 0.2, 0.2, 0.2],
 'FailedGo': [0.2,
  0.2,
  0.3333333333333333,
  0.2,
  0.25,
  0.2,
  0.25,
  0.2,
  0.2,
  0.6],
 'FailedStop': [0.2,
  0.2,
  0.3333333333333333,
  0.2,
  0.25,
  0.2,
  0.25,
  0.2,
  0.2,
  0.2]}

Yep those look similar so I'm confident I am doing the right thing setting up this scorer.